# [AHA! Activity Health Analytics](http://casas.wsu.edu/)
[Center for Advanced Studies of Adaptive Systems (CASAS)](http://casas.wsu.edu/)

[Washington State University](https://wsu.edu)

# L1 Fitbit API

## Learner Objectives
At the conclusion of this lesson, participants should have an understanding of:
* Wearable fitness devices
* Interfacing with the Fitbit API

## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Fitbit API docs](https://dev.fitbit.com/docs/)

## Wearable Sensors
Recently, wearable sensors have increased in popularity as people aspire to be more conscientious of their physical health. Many consumers purchase a pedometer or wearable fitness device in order to track their physical activity (PA), often in pursuit of a goal such as increasing cardiovascular strength, losing weight, or improving overall health. 

### Fitbit
One of the most common wearable device manufacturers is [Fitbit](https://www.fitbit.com/home). Fitbit offers several different wearable devices that include different combinations of the following sensors:
* 3-axis accelerometer (detects movement)
* Altimeter (detects changes in air pressure)
* Optical heart rate monitor
* GPS
* Digital compass
* Ambient light sensor (detects changes in light)

<img src="http://www.fitbit.co.za/wp-content/uploads/2016/10/4067_19.png" width="500">
(image from [http://www.fitbit.co.za/wp-content/uploads/2016/10/4067_19.png](http://www.fitbit.co.za/wp-content/uploads/2016/10/4067_19.png))


### Fitbit Data
Data collected from these sensors are processed by proprietary Fitbit software to compute several metrics related to physical activity, including:
* Steps
* Distance traveled
* Floors climbed
* Calories burned
* Heart rate
* Sleep
* Location

When you buy a Fitbit, you need to [setup a Fitbit account](https://www.fitbit.com/setup). Data collected by the Fitbit device is synced to your Fitbit account, where the data is stored in the cloud. By logging into your Fitbit account on your smartphone or on your computer, you can see aggregated metrics such as daily (or weekly or monthly) steps taken, floors climbed, calories burned, etc. Using the [Fitbit API](https://dev.fitbit.com/), we can download the source data for these aggregate metrics. This data is minute by minute time series data for all 1,440 minutes of the day! In this lesson, we are going to learn how download this fine-grained data. In later lessons, we will visualize and analyze minute by minute Fitbit data for behavior change detection.

Note: you do not need a Fitbit device or an account for this lesson. The Fitbit data used for visualization and analysis will be provided as csv files.

## Fitbit API
An API (application programming interface) provides functions for programmers to use, without needing to know the details of the implementation. As long as we know:
1. The API interface
1. What format the API expects
1. What the API returns

Then we can use an API like Fitbit! Fitbit provides a Web API for accessing data from Fitbit activity trackers, Aria scale, and manually entered logs. Anyone can develop an application to access and modify a Fitbit user's data on their behalf, so long as it complies with Fitbit's API Terms of Use.

Note: The Fitbit API requires *authentication*, meaning an API key that uniquely identifies you and/or your program is used to gain access to the API. Not all APIs require an API key to access the API. For example, the [Google Maps Distance Matrix API](https://developers.google.com/maps/web-services/overview) does not require a key.

If you would like to setup to download your own Fitbit data, follow these instructions from the [Fitbit API docs](https://dev.fitbit.com/docs/):
1. Review the [basics about how the Fitbit Web API works](https://dev.fitbit.com/docs/basics/).
1. [Register your application](https://dev.fitbit.com/apps/new) to get API client credentials. You will need a Fitbit account (free) to register an app.
    * Make sure to enter the following information:
         * OAuth 2.0 Application Type set as Personal (access to intraday time series data (such as Heart Rate or Activity) of the owner of the app only)
         * Callback URL set to [http://127.0.0.1:8080/](http://127.0.0.1:8080/) 
1. Implement an [OAuth 2.0 authorization flow](https://dev.fitbit.com/docs/oauth2) to allow people to give your app permission to access data on their behalf.
1. Make HTTP requests to access data. The different types of data available via the Web API are listed in the navigation. You can also try the [API Explorer](https://apigee.com/me3/embed/console/fitbit).
1. If you have a server app and want to be notified when people have new data available, implement the Subscriptions API.

Step 1 is helpful because it provides more background information about the API. Step 2 requires that you register an app in order to gain access to the API. This step is completed in your browser and does not require any coding.

### Install Python Fitbit Client
We will use the [`python-fitbit`](https://github.com/orcasgit/python-fitbit) Python library to implement step 3 and help with step 4. This will save us alot of time as we won't need to learn the details of the authentication system or constructing API queries using HTTP. 

To download and install the [`python-fitbit`](https://github.com/orcasgit/python-fitbit) library: 
1. Download a zip file of the [`python-fitbit`](https://github.com/orcasgit/python-fitbit) Github repository.
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/fitbit_github_download.png" width="600">
1. Extract the zip file.
1. Open a [command prompt (Windows)](https://www.lifewire.com/how-to-open-command-prompt-2618089) or [terminal (Mac/Linux)](http://www.wikihow.com/Open-a-Terminal-Window-in-Mac).
    * Windows
    <img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/cmd.png" width="400">
    * Mac/Linux
    <img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/terminal.png" width="400">
1. Using [`cd` (change directory) command](https://en.wikipedia.org/wiki/Cd_(command), navigate into the extracted folder.
1. Enter the following commands to install the library and its requirements (you may need to run these commands as the administrator on your machine (e.g. sudo)): 
    1. `pip install -r requirements/base.txt`
    1. `pip install fitbit`
    1. `pip install cherrypy`
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/fitbit_pip_install.png" width="600">

### Gather Authentication Information
Now we need the following information in order to establish an authenticated connection from our Python client program, to our Fitbit app, for a user account (yours). 

#### Fitbit App Info
On the [Fitbit website](https://dev.fitbit.com/), go to [Manage Your Apps](https://dev.fitbit.com/apps), and click on your app. You will see a page that looks similar to the following (our app information has been removed):
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/fitbit_application_page.png" width="500">.

From this page, we will need:
1. OAuth 2.0 Client ID (your app ID)
1. Client (consumer) secret (used by your program as a "password" to connect to your app)

In general, it is considered bad practice to store your API keys in your program (read Google's ["Best practices for securing API keys"](https://support.google.com/googleapi/answer/6310037) page. The first bullet point reads: **"Do not embed API keys directly in code".**). I will store mine in a text file `fitbit_client_keys.txt`. You will need to create this file yourself.

#### User Authentication
Now we need to authenticate our user account with our Fitbit app in order to download our data. The `gather_keys_oauth2.py` file is included in the the `python-fitbit` Github repository. You can either run `gather_keys_oauth2.py` directly in Python specifying your client ID and client secret as command line arguments, or import it and instantiate a `OAuth2Server` object passing your client ID and client secret as constructor arguments. Since this is running in Jupyter Notebook, it is more convenient to take the latter approach. 

In [99]:
import files.gather_keys_oauth2 as gather

# storing Fitbit app info external to this program
in_file = open(r"files\fitbit_client_keys.txt", "r")
app_info = in_file.readlines()
CLIENT_ID = app_info[0].strip()
CLIENT_SECRET = app_info[1].strip()
server = gather.OAuth2Server(CLIENT_ID, CLIENT_SECRET)

Now, we need to log in to our account to authorize our app to access our data. When you run the next code cell, the Fitbit login page should appear.
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/fitbit_login.png" width="400">.

Log in with the same user name that is the owner of your app.

In [100]:
server.browser_authorize()

[26/May/2017:15:14:30] ENGINE Listening for SIGTERM.
[26/May/2017:15:14:30] ENGINE Bus STARTING
[26/May/2017:15:14:30] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[26/May/2017:15:14:30] ENGINE Started monitor thread 'Autoreloader'.
[26/May/2017:15:14:30] ENGINE Started monitor thread '_TimeoutMonitor'.
[26/May/2017:15:14:30] ENGINE Serving on http://127.0.0.1:8080
[26/May/2017:15:14:30] ENGINE Bus STARTED


127.0.0.1 - - [26/May/2017:15:14:32] "GET /?code=ea0d6f3e953c9832926c371ce51fe18add177d87&state=fHVplZdIPQJX1zHB8kUQZkkGf8M4mz HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"


[26/May/2017:15:14:33] ENGINE Bus STOPPING
[26/May/2017:15:14:42] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[26/May/2017:15:14:42] ENGINE Stopped thread 'Autoreloader'.
[26/May/2017:15:14:42] ENGINE Stopped thread '_TimeoutMonitor'.
[26/May/2017:15:14:42] ENGINE Removed handler for console events.
[26/May/2017:15:14:42] ENGINE Bus STOPPED
[26/May/2017:15:14:42] ENGINE Bus EXITING
[26/May/2017:15:14:42] ENGINE Waiting for child threads to terminate...
[26/May/2017:15:14:42] ENGINE Bus EXITED
[26/May/2017:15:14:42] ENGINE Waiting for thread Thread-50.


If all goes well, you should see the following webpage (the website listed as the callback URL: [http://127.0.0.1:8080/](http://127.0.0.1:8080/)): 
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/fitbit_cherrypy_8080.png" width="500">

Let's extract the user authentication information `python-fitbit` needs to query data from Fitbit:
* User ID associated with our user name
* Oauth 2.0 access token
    Note: An access token intentionally is short lived (eight-hour lifetime by default).
* Oauth 2.0 refresh token
    * Note: a refresh token is used to retrieve a new access token. A refresh token does not expire until it is used. A refresh token can only be used once, as a new refresh token is returned with the new access token. If you want to learn more about refresh tokens, read the [Fitbit Oauth 2.0 documentation](https://dev.fitbit.com/docs/oauth2/#refreshing-tokens).

In [101]:
USER_ID = server.fitbit.client.session.token['user_id']
ACCESS_TOKEN =  server.fitbit.client.session.token['access_token']
REFRESH_TOKEN =  server.fitbit.client.session.token['refresh_token']

Now we can authorize ourselves with the Fitbit API and Oauth2.0 by instantiating a `python-fitbit` `Fitbit` object. This object will be our interface to the Fitbit API.

In [102]:
import fitbit

authd_client = fitbit.Fitbit(USER_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

### User Information
At last! We are ready to collect our data from the Fitbit API. Let's grab our display name. This could be useful for naming an output csv file later.

In [103]:
user_stats = authd_client.user_profile_get('-')
display_name = user_stats["user"]["displayName"]
print(display_name)

Gina


Fitbit API responses are in [JSON](http://www.json.org/) format. `python-fitbit` returns the JSON responses as Python dictionaries which makes the response straightforward to parse.